In [1]:
import numpy as np
import numpy.linalg as LA

In [36]:
# Originalne tacke i tacke projekcije
x1 = np.array([958, 38, 1.])
y1 = np.array([933, 33, 1.])
x2 = np.array([1117, 111, 1.])
y2 = np.array([1027, 132, 1.])
x3 = np.array([874, 285, 1.])
y3 = np.array([692, 223, 1.])
x4 = np.array([707, 218, 1.])
y4 = np.array([595, 123, 1.])
x9 = np.array([292, 569, 1.])
y9 = np.array([272, 360, 1.])
x10 = np.array([770, 969, 1.])
y10 = np.array([432, 814, 1.])
x11 = np.array([770, 1465, 1.])
y11 = np.array([414, 1284, 1.])
x12 = np.array([317, 1057, 1.])
y12 = np.array([258, 818, 1.])

# Vektori tih tacaka, bez normalizacije
tacke_1 = np.array([x1, x2, x3, x4, x9, x10, x11, x12])
tacke_2 = np.array([y1, y2, y3, y4, y9, y10, y11, y12])

In [27]:
# Jadnacina y^T*F*x=0
def jed(x, y):
  return [x[0]*y[0], x[1]*y[0], x[2]*y[0], x[0]*y[1], x[1]*y[1], x[2]*y[1], x[0]*y[2], x[1]*y[2], x[2]*y[2]]

In [28]:
# Funkcija za prebacivanje u afine koordinate
def afinize(x):
  return x / x[-1]

In [29]:
# Matrica koja predstavlja 8 jednacina dobijenih iz korespodencija
matrix = np.zeros((8, 9))

for i in range(8):
  matrix[i] = jed(tacke_1[i], tacke_2[i])


In [30]:
# Konstruisanje fundamentalne matrice
SVD = LA.svd(matrix)

F = SVD[-1][-1].reshape(3, 3)

print(F)

[[ 4.54020258e-07 -7.42529108e-07 -1.86611244e-03]
 [-1.99099442e-07  2.11775653e-07  2.77795603e-03]
 [ 3.96239362e-04 -2.72717967e-03  9.99990603e-01]]


In [31]:
# Testiranje da li vazi uslov y^T*F*x = 0
# i da li je determinanta fundamentalne matrice
# bliska nuli

def test(x, y, F):
  x = np.array(x)
  y = np.array(y)
  return y @ F @ x

print("Testiranje:")

for i in range(8):
  if test(tacke_1[i], tacke_2[i], F) != 0:
    print(test(tacke_1[i], tacke_2[i], F))

print()
print("Determinanta fundamentalne matrice:")
print(LA.det(F))

Testiranje:
2.220446049250313e-16
2.098321516541546e-14
1.021405182655144e-14
3.247402347028583e-15
1.3766765505351941e-14
-1.554312234475219e-14
-5.5067062021407764e-14
-4.618527782440651e-14

Determinanta fundamentalne matrice:
1.713967417960357e-12


In [32]:
# S, V, D dekompozicija fundamentalne matrice
SVDF = LA.svd(F)
U, D, V = SVDF

# Odredjivanje prvog i drugog epipola
e1 = V[-1]
e1 = afinize(e1)

e2 = U[:, -1]
e2 = afinize(e2)

print()
print("Koordinate prvod epipola:")
print(e1)
print("Koordinate drugog epipola:")
print(e2)



Koordinate prvod epipola:
[1.04078394e+04 1.87885919e+03 1.00000000e+00]
Koordinate drugog epipola:
[-4.38831707e+03 -3.30785789e+03  1.00000000e+00]


In [33]:
# Postizanje uslova det(F) = 0

D1 = np.diag([1, 1, 0]) @ D
D1 = np.diag(D1)

F1 = U @ D1 @ V

print(LA.det(F1))

4.855996621224824e-28


In [38]:
# Preostale vidljive tacke

x6 = np.array([1094, 536, 1.])
y6 = np.array([980, 535, 1.])
x7 = np.array([862, 729, 1.])
y7 = np.array([652, 638, 1.])
x8 = np.array([710, 648, 1.])
y8 = np.array([567, 532, 1.])
x14 = np.array([1487, 598, 1.])
y14 = np.array([1303, 700, 1.])
x15 = np.array([1462, 1079, 1.])
y15 = np.array([1257, 1165, 1.])
y13 = np.array([1077, 269, 1.])

In [40]:
# Odredjivanje skrivenih tacaka
def cross(a, b, c, d, e, f, g, h, i, j):
  return np.cross(
      np.cross(np.cross(np.cross(a, b), np.cross(c, d)), e),
      np.cross(np.cross(np.cross(f, g), np.cross(h, i)), j)
  )

# Nevidljive tacke prve projekcije
x5 = cross(x4, x8, x6, x2, x1, x1, x4, x3, x2, x8)

x13 = cross(x9, x10, x11, x12, x14, x11, x15, x10, x14,  x9)

x16 = cross(x10, x14, x11, x15, x12, x9, x10, x11, x12, x15)

# Nevidljive tacke druge projekcije
y5 = cross(y4, y8, y6, y2, y1, y1, y4, y3, y2, y8)

y16 = cross(y10, y14, y11, y15, y12, y9, y10, y11, y12, y15)

In [55]:
# Triangulacija

# Kanonska matrica kamere
T1 = np.hstack([np.eye(3), np.zeros(3).reshape(3, 1)])

# Matrica vektorskog mnozenja
vec = lambda p: np.array([[  0,   -p[2],  p[1]],
                          [ p[2],   0,   -p[0]],
                          [-p[1],  p[0],   0  ]])

# Matrica drugog epipola
E2 = vec(e2)

# Druga matrica kamere
T2 = np.hstack([E2 @ F1, e2.reshape(3, 1)])

# Za svaku tacku po sistem od cetiri jednacine
# sa cetiri homogene nepoznate, mada mogu i tri
jednacine = lambda xx, yy: np.array([ xx[1]*T1[2] - xx[2]*T1[1],
                                       -xx[0]*T1[2] + xx[2]*T1[0],
                                        yy[1]*T2[2] - yy[2]*T2[1],
                                       -yy[0]*T2[2] + yy[2]*T2[0]])

# Prebacivanje u afine koordinate oblika (x, y)
def toAffine(x):
  return afinize(x)[:-1]

# Funkcija koja vraca 3D koordinate rekonstruisane tacke
def ThreeD(x, y):
  return toAffine(LA.svd(jednacine(x, y))[-1][-1])

image_1 = np.array([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16])
image_2 = np.array([y1, y2, y3, y4, y5, y6, y7, y8, y9, y10, y11, y12, y13, y14, y15, y16])

# Vektor rekonstruisanih tacaka
reconstructed = np.array([ThreeD(x, y) for x, y in zip(image_1, image_2)])

def multiplie(x, n):
  return np.diag([1, 1, n]) @ x

# Mnozimo z koordinatu jer nije radjena normalizacija
reconstructed400 = np.array([multiplie(x, 400) for x in reconstructed])

iviceMala = np.array([[1, 2], [2, 3], [3, 4], [4, 1],
                      [5, 6], [6, 7], [7, 8], [8, 5],
                      [1, 5], [2, 6], [3, 7], [4, 8]])

iviceVelika = np.array([[ 9, 10], [10, 11], [11, 12], [12,  9], 
                        [13, 14], [14, 15], [15, 16], [16, 13],
                        [ 9, 13], [10, 14], [11, 15], [12, 16]])